In [ ]:
import os

import numpy as np
import illustris_python as il
%pylab inline

import constants

In [ ]:
base_path = os.path.join(constants.TNG_DIR_BASE, 'output')
to_msun_conversion_fac = 1e10 / 0.6774

subhalos = il.groupcat.loadSubhalos(base_path, 30, fields=['SubhaloMassType'])

In [ ]:
dm_mass, stellar_mass = subhalos[:, 1], subhalos[:, 4]
dm_mass *= to_msun_conversion_fac
stellar_mass *= to_msun_conversion_fac

In [ ]:
print(np.sum(dm_mass == 0))
print(np.sum(stellar_mass == 0))

In [ ]:
mask = np.logical_or(dm_mass == 0, stellar_mass == 0)
dm_mass = dm_mass[~mask]
stellar_mass = stellar_mass[~mask]

log10_dm_mass = np.log10(dm_mass)
log10_stellar_mass = np.log10(stellar_mass)

In [ ]:
plt.hist(log10_dm_mass)
plt.show()
plt.hist(log10_stellar_mass)

In [ ]:
universemachine_z2_3 = np.loadtxt(os.path.join(constants.AUX_DIR_BASE, 'umachine-dr1/data/smhm/averages/hm_averages_a0.303685.dat'))
umachine_smass_bin_edges = universemachine_z2_3[:, 64:66]
smass_bin_center = universemachine_z2_3[:, 0]

In [ ]:
for edges, center in zip(umachine_smass_bin_edges, smass_bin_center):
    print(edges, center)
    assert np.isclose(center, (edges[0] + edges[1]) / 2)

In [ ]:
# for each stellar mass bin, compute average halo (DM) mass in linspace for TNG
tng_hmass = []
for edges in umachine_smass_bin_edges:
    mask = np.logical_and(stellar_mass >= 10**edges[0], stellar_mass < 10**edges[1])
    tng_hmass.append(np.mean(dm_mass[mask]))
    
tng_log10_hmass = np.log10(np.array(tng_hmass))

In [ ]:
plt.plot(smass_bin_center, tng_log10_hmass)

In [ ]:
output_arr = np.array([smass_bin_center, tng_log10_hmass]).T
print(output_arr.shape)
np.save(os.path.join(constants.AUX_DIR_BASE, 'tng100-1_z2.3_log10_smhm.npy'), output_arr)